# 머신러닝 with Python 
## 09 앙상블 학습 보팅 연습2- wine data set

##### 노트를 활용해 직접 다른 데이터로 해보았다.

책 p.259~ (원본 데이터 - 붓꽃 분류, 이번 사용 데이터 와인 데이터), 일부는 랜덤포레스트 실습 부분 참조

---

하단부에서는 세 클래시파이어(로지스틱 회귀분석, 서포트 벡터 머신, 가우시안 나이브 베이즈) 중 하나씩 빼보면서 실험해 보았다.

> 배운점1: 세 분류기 합친 모델의 정확도 >= 세 분류기 중 하나가 빠진 모델의 정확도 

> 배운점2 : a b 분류기를 함께 보팅으로 사용했을 때의 정확도 >= a 혹은 b 분류기의 정확도 중 **낮은 정확도** 

### 데이터 불러오기

In [1]:
from sklearn import datasets 

In [2]:
raw_wine = datasets.load_wine() # 와인 데이터 가져오기

In [3]:
# 데이터 살펴보기

raw_wine

{'data': array([[1.423e+01, 1.710e+00, 2.430e+00, ..., 1.040e+00, 3.920e+00,
         1.065e+03],
        [1.320e+01, 1.780e+00, 2.140e+00, ..., 1.050e+00, 3.400e+00,
         1.050e+03],
        [1.316e+01, 2.360e+00, 2.670e+00, ..., 1.030e+00, 3.170e+00,
         1.185e+03],
        ...,
        [1.327e+01, 4.280e+00, 2.260e+00, ..., 5.900e-01, 1.560e+00,
         8.350e+02],
        [1.317e+01, 2.590e+00, 2.370e+00, ..., 6.000e-01, 1.620e+00,
         8.400e+02],
        [1.413e+01, 4.100e+00, 2.740e+00, ..., 6.100e-01, 1.600e+00,
         5.600e+02]]),
 'target': array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1

### 피처, 타깃 데이터 지정

보통 train data는 행렬이므로 대문자 X, 예측되어야 할 값은 스칼라로 소문자 y가 된다는 것을 들은게 생각이 나부렀다.}

In [4]:
X = raw_wine.data
y = raw_wine.target

### 트레이닝/테스트 데이터 분할

In [5]:
from sklearn.model_selection import train_test_split              # 분할을 위해 필요한 함수
X_tn, X_te, y_tn, y_te = train_test_split(X, y, random_state = 0) # 분리. randomstate는 고정

### 데이터 표준화

In [6]:
from sklearn.preprocessing import StandardScaler # 데이터 표준화를 위한 함수
std_scale = StandardScaler()                     # 표준화 스케일러 지정
std_scale.fit(X_tn)                              # 트레이닝 피처를 기준으로 표준화를 적합시키기


X_tn_std = std_scale.transform(X_tn) 
X_te_std = std_scale.transform(X_te)             # 트레인, 테스트 데이터 각각 적합시킨 표준화에 맞게 변형

### 데이터 학습 - 보팅 학습

이번 실습에서는 로지스틱 회귀분석, 가우시안 나이브 베이즈, 서포트 벡터 머신을 사용한다.

In [7]:
from sklearn.linear_model import LogisticRegression # 로지스틱 회귀분석
from sklearn import svm                             # support vector machine
from sklearn.naive_bayes import GaussianNB          # Gussian Naive Bayes
from sklearn.ensemble import VotingClassifier       # 분류 문제이기 때문이 VotingClassifier를 사용. 회귀문제라면 VotingRegressor를 사용한다.

clf1 = LogisticRegression(multi_class = 'multinomial', 
                          random_state = 1)         # 첫 번째 모형을 로지스틱 회귀분석이라고 정함
clf2 = svm.SVC(kernel = 'linear', 
              random_state = 1)                     # 두 번째 모형을 서포트 벡터 머신 모형이라고 정함
clf3 = GaussianNB()                                 # 세 번째 모형은 가우시안 나이브 베이즈

clf_voting = VotingClassifier(estimators = [        # estimators : 미리 만든 세 가지 모형을 의미
                                    ('lr', clf1),
                                    ('svm', clf2),
                                    ('gnb', clf3),
                                                ],
                             voting = 'hard',       # voting : hard(default)=과반수가 넘는 라벨, soft=확률이 가장 높은 라벨
                             weights = [1, 1, 1])   # weights : 각 모형의 비율. 중요도 가중치를 어떻게 줄지인 것으로 예상됨. 

clf_voting.fit(X_tn_std, y_tn)                      # 만들어진 모형에 표준화된 트레이닝 피처 데이터와 트레이닝 타깃 데이터를 넣고 적합시킨다.

VotingClassifier(estimators=[('lr',
                              LogisticRegression(multi_class='multinomial',
                                                 random_state=1)),
                             ('svm', SVC(kernel='linear', random_state=1)),
                             ('gnb', GaussianNB())],
                 weights=[1, 1, 1])

### 데이터 예측

In [8]:
pred_voting = clf_voting.predict(X_te_std)          # std된 테스트 피처 데이터를 넣고 실행
print(pred_voting)

[0 2 1 0 1 1 0 2 1 1 2 2 0 1 2 1 0 0 1 0 1 0 0 1 1 1 1 1 1 2 0 0 1 0 0 0 2
 1 1 2 0 0 1 1 1]


### 정확도 평가

In [9]:
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_te, pred_voting)        # 실제값과 예측값을 넣음
print(accuracy)

1.0


...?

### confusion matrix 확인

In [10]:
from sklearn.metrics import confusion_matrix
conf_matrix = confusion_matrix(y_te, pred_voting)
print(conf_matrix)

[[16  0  0]
 [ 0 21  0]
 [ 0  0  8]]


### 분류 리포트 확인

In [11]:
from sklearn.metrics import classification_report
class_report = classification_report(y_te, pred_voting)
print(class_report)

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        16
           1       1.00      1.00      1.00        21
           2       1.00      1.00      1.00         8

    accuracy                           1.00        45
   macro avg       1.00      1.00      1.00        45
weighted avg       1.00      1.00      1.00        45



말이 되냐구요... 이러면 안되는 거잖아...

---

아 모델 보팅에 놓는 순서에 대해 얘기하려다가 별 상관없겠다고 생각듬 그이유: 같은 데이터로 하는 voting임.

순차적으로 입출력값을 받는 것도 아님.

# 실험1: 모델 하나씩 빼보기

모델을 하나씩 빼고 정확도를 보기로 함.

## 모델 하나씩 빼보기 (- 가우시안 나이브 베이즈)

In [12]:
from sklearn.linear_model import LogisticRegression # 로지스틱 회귀분석
from sklearn import svm                             # support vector machine
from sklearn.naive_bayes import GaussianNB          # Gussian Naive Bayes
from sklearn.ensemble import VotingClassifier       # 분류 문제이기 때문이 VotingClassifier를 사용. 회귀문제라면 VotingRegressor를 사용한다.

clf1 = LogisticRegression(multi_class = 'multinomial', 
                          random_state = 1)         # 첫 번째 모형을 로지스틱 회귀분석이라고 정함
clf2 = svm.SVC(kernel = 'linear', 
              random_state = 1)                     # 두 번째 모형을 서포트 벡터 머신 모형이라고 정함
# clf3 = GaussianNB()                                 # 세 번째 모형은 가우시안 나이브 베이즈

clf_voting = VotingClassifier(estimators = [        # estimators : 미리 만든 세 가지 모형을 의미
                                    ('lr', clf1),
                                    ('svm', clf2)
                                                ],
                             voting = 'hard',       # voting : hard(default)=과반수가 넘는 라벨, soft=확률이 가장 높은 라벨
                             weights = [1, 1])   # weights : 각 모형의 비율. 중요도 가중치를 어떻게 줄지인 것으로 예상됨. 

clf_voting.fit(X_tn_std, y_tn)                      # 만들어진 모형에 표준화된 트레이닝 피처 데이터와 트레이닝 타깃 데이터를 넣고 적합시킨다.

VotingClassifier(estimators=[('lr',
                              LogisticRegression(multi_class='multinomial',
                                                 random_state=1)),
                             ('svm', SVC(kernel='linear', random_state=1))],
                 weights=[1, 1])

### 데이터 예측

In [13]:
pred_voting = clf_voting.predict(X_te_std)          # std된 테스트 피처 데이터를 넣고 실행
print(pred_voting)

[0 2 1 0 1 1 0 2 1 1 2 2 0 1 2 1 0 0 1 0 1 0 0 1 1 1 1 1 1 2 0 0 1 0 0 0 2
 1 1 2 0 0 1 1 1]


### 정확도 평가

In [15]:
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_te, pred_voting)        # 실제값과 예측값을 넣음
print(accuracy)

1.0


...?

### confusion matrix 확인

In [16]:
from sklearn.metrics import confusion_matrix
conf_matrix = confusion_matrix(y_te, pred_voting)
print(conf_matrix)

[[16  0  0]
 [ 0 21  0]
 [ 0  0  8]]


### 분류 리포트 확인

In [17]:
from sklearn.metrics import classification_report
class_report = classification_report(y_te, pred_voting)
print(class_report)

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        16
           1       1.00      1.00      1.00        21
           2       1.00      1.00      1.00         8

    accuracy                           1.00        45
   macro avg       1.00      1.00      1.00        45
weighted avg       1.00      1.00      1.00        45



말이 되냐구요... 이러면 안되는 거잖아...

## 모델 하나씩 빼보기 (- 서포트 벡터 머신)

In [18]:
from sklearn.linear_model import LogisticRegression # 로지스틱 회귀분석
from sklearn import svm                             # support vector machine
from sklearn.naive_bayes import GaussianNB          # Gussian Naive Bayes
from sklearn.ensemble import VotingClassifier       # 분류 문제이기 때문이 VotingClassifier를 사용. 회귀문제라면 VotingRegressor를 사용한다.

clf1 = LogisticRegression(multi_class = 'multinomial', 
                          random_state = 1)         # 첫 번째 모형을 로지스틱 회귀분석이라고 정함
# clf2 = svm.SVC(kernel = 'linear', 
#               random_state = 1)                     # 두 번째 모형을 서포트 벡터 머신 모형이라고 정함
clf3 = GaussianNB()                                 # 세 번째 모형은 가우시안 나이브 베이즈

clf_voting = VotingClassifier(estimators = [        # estimators : 미리 만든 세 가지 모형을 의미
                                    ('lr', clf1),
                                    ('gnb', clf3)
                                                ],
                             voting = 'hard',       # voting : hard(default)=과반수가 넘는 라벨, soft=확률이 가장 높은 라벨
                             weights = [1, 1])   # weights : 각 모형의 비율. 중요도 가중치를 어떻게 줄지인 것으로 예상됨. 

clf_voting.fit(X_tn_std, y_tn)                      # 만들어진 모형에 표준화된 트레이닝 피처 데이터와 트레이닝 타깃 데이터를 넣고 적합시킨다.

VotingClassifier(estimators=[('lr',
                              LogisticRegression(multi_class='multinomial',
                                                 random_state=1)),
                             ('gnb', GaussianNB())],
                 weights=[1, 1])

### 데이터 예측

In [19]:
pred_voting = clf_voting.predict(X_te_std)          # std된 테스트 피처 데이터를 넣고 실행
print(pred_voting)

[0 2 1 0 1 1 0 2 1 1 2 2 0 0 2 1 0 0 1 0 0 0 0 1 1 1 1 1 1 2 0 0 1 0 0 0 2
 1 1 2 0 0 1 1 1]


### 정확도 평가

In [20]:
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_te, pred_voting)        # 실제값과 예측값을 넣음
print(accuracy)

0.9555555555555556


서포트 벡터 머신을 뺐더니 정확도가 하락했다.

컨퓨전 매트릭스를 확인했더니 클래스 1를 0으로 분류한 오류 몇 개가 생겼다.

### confusion matrix 확인

In [21]:
from sklearn.metrics import confusion_matrix
conf_matrix = confusion_matrix(y_te, pred_voting)
print(conf_matrix)

[[16  0  0]
 [ 2 19  0]
 [ 0  0  8]]


### 분류 리포트 확인

In [23]:
from sklearn.metrics import classification_report
class_report = classification_report(y_te, pred_voting)
print(class_report)

              precision    recall  f1-score   support

           0       0.89      1.00      0.94        16
           1       1.00      0.90      0.95        21
           2       1.00      1.00      1.00         8

    accuracy                           0.96        45
   macro avg       0.96      0.97      0.96        45
weighted avg       0.96      0.96      0.96        45



재밌다. 서포트 벡터 머신이 귀한 역할을 했어야 했나보다.

## 모델 하나씩 빼보기 (- 로지스틱 리그레션)

In [32]:
from sklearn.linear_model import LogisticRegression # 로지스틱 회귀분석
from sklearn import svm                             # support vector machine
from sklearn.naive_bayes import GaussianNB          # Gussian Naive Bayes
from sklearn.ensemble import VotingClassifier       # 분류 문제이기 때문이 VotingClassifier를 사용. 회귀문제라면 VotingRegressor를 사용한다.

# clf1 = LogisticRegression(multi_class = 'multinomial', 
#                           random_state = 1)         # 첫 번째 모형을 로지스틱 회귀분석이라고 정함
clf2 = svm.SVC(kernel = 'linear', 
              random_state = 1)                     # 두 번째 모형을 서포트 벡터 머신 모형이라고 정함
clf3 = GaussianNB()                                 # 세 번째 모형은 가우시안 나이브 베이즈

clf_voting = VotingClassifier(estimators = [        # estimators : 미리 만든 세 가지 모형을 의미
                                    ('svm', clf2),
                                    ('gnb', clf3)
                                                ],
                             voting = 'hard',       # voting : hard(default)=과반수가 넘는 라벨, soft=확률이 가장 높은 라벨
                             weights = [1, 1])   # weights : 각 모형의 비율. 중요도 가중치를 어떻게 줄지인 것으로 예상됨. 

clf_voting.fit(X_tn_std, y_tn)                      # 만들어진 모형에 표준화된 트레이닝 피처 데이터와 트레이닝 타깃 데이터를 넣고 적합시킨다.

VotingClassifier(estimators=[('svm', SVC(kernel='linear', random_state=1)),
                             ('gnb', GaussianNB())],
                 weights=[1, 1])

### 데이터 예측

In [33]:
pred_voting = clf_voting.predict(X_te_std)          # std된 테스트 피처 데이터를 넣고 실행
print(pred_voting)

[0 2 1 0 1 1 0 2 1 1 2 2 0 0 2 1 0 0 1 0 0 0 0 1 1 1 1 1 1 2 0 0 1 0 0 0 2
 1 1 2 0 0 1 1 1]


### 정확도 평가

In [34]:
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_te, pred_voting)        # 실제값과 예측값을 넣음
print(accuracy)

0.9555555555555556


### confusion matrix 확인

In [35]:
from sklearn.metrics import confusion_matrix
conf_matrix = confusion_matrix(y_te, pred_voting)
print(conf_matrix)

[[16  0  0]
 [ 2 19  0]
 [ 0  0  8]]


### 분류 리포트 확인

In [36]:
from sklearn.metrics import classification_report
class_report = classification_report(y_te, pred_voting)
print(class_report)

              precision    recall  f1-score   support

           0       0.89      1.00      0.94        16
           1       1.00      0.90      0.95        21
           2       1.00      1.00      1.00         8

    accuracy                           0.96        45
   macro avg       0.96      0.97      0.96        45
weighted avg       0.96      0.96      0.96        45



## 결과 리뷰

서포트 벡터 머신을 뺐을 때와 비슷한 정확도를 보여주었다.

lr + svm $\rightarrow$ 1.0

lr + gnb $\rightarrow$ 0.9555

svm + gnb $\rightarrow$ 0.9555

이것만 보고 뭐가 더 좋다고 하기는 어려우니 한 가지 모델로만 해보기도 해보자.

# 실험2 각 모델 하나씩으로만 해보기

## logistic regreesion 단독

In [37]:
from sklearn.linear_model import LogisticRegression # 로지스틱 회귀분석
from sklearn import svm                             # support vector machine
from sklearn.naive_bayes import GaussianNB          # Gussian Naive Bayes
from sklearn.ensemble import VotingClassifier       # 분류 문제이기 때문이 VotingClassifier를 사용. 회귀문제라면 VotingRegressor를 사용한다.

clf1 = LogisticRegression(multi_class = 'multinomial', 
                          random_state = 1)         # 첫 번째 모형을 로지스틱 회귀분석이라고 정함
# clf2 = svm.SVC(kernel = 'linear', 
#               random_state = 1)                     # 두 번째 모형을 서포트 벡터 머신 모형이라고 정함
# clf3 = GaussianNB()                                 # 세 번째 모형은 가우시안 나이브 베이즈

clf_voting = VotingClassifier(estimators = [('lr', clf1)],
                             voting = 'hard',       # voting : hard(default)=과반수가 넘는 라벨, soft=확률이 가장 높은 라벨
                             weights = [1])   # weights : 각 모형의 비율. 중요도 가중치를 어떻게 줄지인 것으로 예상됨. 

clf_voting.fit(X_tn_std, y_tn)                      # 만들어진 모형에 표준화된 트레이닝 피처 데이터와 트레이닝 타깃 데이터를 넣고 적합시킨다.

VotingClassifier(estimators=[('lr',
                              LogisticRegression(multi_class='multinomial',
                                                 random_state=1))],
                 weights=[1])

### 데이터 예측

In [38]:
pred_voting = clf_voting.predict(X_te_std)          # std된 테스트 피처 데이터를 넣고 실행
print(pred_voting)

[0 2 1 0 1 1 0 2 1 1 2 2 0 1 2 1 0 0 1 0 1 0 0 1 1 1 1 1 1 2 0 0 1 0 0 0 2
 1 1 2 0 0 1 1 1]


### 정확도 평가

In [39]:
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_te, pred_voting)        # 실제값과 예측값을 넣음
print(accuracy)

1.0


### confusion matrix 확인

In [40]:
from sklearn.metrics import confusion_matrix
conf_matrix = confusion_matrix(y_te, pred_voting)
print(conf_matrix)

[[16  0  0]
 [ 0 21  0]
 [ 0  0  8]]


### 분류 리포트 확인

In [41]:
from sklearn.metrics import classification_report
class_report = classification_report(y_te, pred_voting)
print(class_report)

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        16
           1       1.00      1.00      1.00        21
           2       1.00      1.00      1.00         8

    accuracy                           1.00        45
   macro avg       1.00      1.00      1.00        45
weighted avg       1.00      1.00      1.00        45



## SVM 단독

In [42]:
from sklearn.linear_model import LogisticRegression # 로지스틱 회귀분석
from sklearn import svm                             # support vector machine
from sklearn.naive_bayes import GaussianNB          # Gussian Naive Bayes
from sklearn.ensemble import VotingClassifier       # 분류 문제이기 때문이 VotingClassifier를 사용. 회귀문제라면 VotingRegressor를 사용한다.

# clf1 = LogisticRegression(multi_class = 'multinomial', 
#                           random_state = 1)         # 첫 번째 모형을 로지스틱 회귀분석이라고 정함
clf2 = svm.SVC(kernel = 'linear', 
              random_state = 1)                     # 두 번째 모형을 서포트 벡터 머신 모형이라고 정함
# clf3 = GaussianNB()                                 # 세 번째 모형은 가우시안 나이브 베이즈

clf_voting = VotingClassifier(estimators = [('svm', clf2)],
                             voting = 'hard',       # voting : hard(default)=과반수가 넘는 라벨, soft=확률이 가장 높은 라벨
                             weights = [1])   # weights : 각 모형의 비율. 중요도 가중치를 어떻게 줄지인 것으로 예상됨. 

clf_voting.fit(X_tn_std, y_tn)                      # 만들어진 모형에 표준화된 트레이닝 피처 데이터와 트레이닝 타깃 데이터를 넣고 적합시킨다.

VotingClassifier(estimators=[('svm', SVC(kernel='linear', random_state=1))],
                 weights=[1])

### 데이터 예측

In [43]:
pred_voting = clf_voting.predict(X_te_std)          # std된 테스트 피처 데이터를 넣고 실행
print(pred_voting)

[0 2 1 0 1 1 0 2 1 1 2 2 0 1 2 1 0 0 1 0 1 0 0 1 1 1 1 1 1 2 0 0 1 0 0 0 2
 1 1 2 0 0 1 1 1]


### 정확도 평가

In [44]:
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_te, pred_voting)        # 실제값과 예측값을 넣음
print(accuracy)

1.0


### confusion matrix 확인

In [45]:
from sklearn.metrics import confusion_matrix
conf_matrix = confusion_matrix(y_te, pred_voting)
print(conf_matrix)

[[16  0  0]
 [ 0 21  0]
 [ 0  0  8]]


### 분류 리포트 확인

In [46]:
from sklearn.metrics import classification_report
class_report = classification_report(y_te, pred_voting)
print(class_report)

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        16
           1       1.00      1.00      1.00        21
           2       1.00      1.00      1.00         8

    accuracy                           1.00        45
   macro avg       1.00      1.00      1.00        45
weighted avg       1.00      1.00      1.00        45



## Gaussian NB 단독

In [47]:
from sklearn.linear_model import LogisticRegression # 로지스틱 회귀분석
from sklearn import svm                             # support vector machine
from sklearn.naive_bayes import GaussianNB          # Gussian Naive Bayes
from sklearn.ensemble import VotingClassifier       # 분류 문제이기 때문이 VotingClassifier를 사용. 회귀문제라면 VotingRegressor를 사용한다.

# clf1 = LogisticRegression(multi_class = 'multinomial', 
#                           random_state = 1)         # 첫 번째 모형을 로지스틱 회귀분석이라고 정함
# clf2 = svm.SVC(kernel = 'linear', 
#               random_state = 1)                     # 두 번째 모형을 서포트 벡터 머신 모형이라고 정함
clf3 = GaussianNB()                                 # 세 번째 모형은 가우시안 나이브 베이즈

clf_voting = VotingClassifier(estimators = [('gnb', clf3)],
                             voting = 'hard',       # voting : hard(default)=과반수가 넘는 라벨, soft=확률이 가장 높은 라벨
                             weights = [1])   # weights : 각 모형의 비율. 중요도 가중치를 어떻게 줄지인 것으로 예상됨. 

clf_voting.fit(X_tn_std, y_tn)                      # 만들어진 모형에 표준화된 트레이닝 피처 데이터와 트레이닝 타깃 데이터를 넣고 적합시킨다.

VotingClassifier(estimators=[('gnb', GaussianNB())], weights=[1])

### 데이터 예측

In [48]:
pred_voting = clf_voting.predict(X_te_std)          # std된 테스트 피처 데이터를 넣고 실행
print(pred_voting)

[0 2 1 0 1 1 0 2 1 1 2 2 0 0 2 1 0 0 2 0 0 0 0 1 1 1 1 1 1 2 0 0 1 0 0 0 2
 1 1 2 0 0 1 1 1]


### 정확도 평가

In [49]:
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_te, pred_voting)        # 실제값과 예측값을 넣음
print(accuracy)

0.9333333333333333


### confusion matrix 확인

In [50]:
from sklearn.metrics import confusion_matrix
conf_matrix = confusion_matrix(y_te, pred_voting)
print(conf_matrix)

[[16  0  0]
 [ 2 18  1]
 [ 0  0  8]]


### 분류 리포트 확인

In [51]:
from sklearn.metrics import classification_report
class_report = classification_report(y_te, pred_voting)
print(class_report)

              precision    recall  f1-score   support

           0       0.89      1.00      0.94        16
           1       1.00      0.86      0.92        21
           2       0.89      1.00      0.94         8

    accuracy                           0.93        45
   macro avg       0.93      0.95      0.94        45
weighted avg       0.94      0.93      0.93        45



## 결과 리뷰

lr $\rightarrow$ 1.0

svm $\rightarrow$ 1.0

gnb $\rightarrow$ 0.9333

재밌는 결과가 나왔다. 가우시안 나이브 베이즈를 사용했을 때의 정확도만 0.9555 이하였다.

즉 첫번째 실험에서 gnb를 함께 했을 때의 정확도가 낮은 이유가 설명이 되는 부분이었다. 

실제 수식에서도 단독 모델로만 했을 때보다 정확도가 높게 나올 수 밖에 없는게 증명이 되어있었는데, 이 부분을 실제로 확인할 수 있었다.

